# IBM Data Science Capstone Project

## Coursera Course 9

### This notebook contains the code for the execution of the capstone project


In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Poject Course")

Hello Capstone Poject Course


In [3]:
pd.__version__

'0.24.1'

#  PART 1: SEGMENTING AND CLUSTERING NEIGHBOURHOODS IN TORONTO
## Scraping the Wikipedia Page
### Using BeautifulSoup

In [126]:
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df_toronto = pd.read_html(str(table))[0]
postcode = df["Postcode"].tolist()
borough = df["Borough"].tolist()
neighbourhood = df["Neighbourhood"].tolist()
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Removing the 'Not assigned' cells in Borough

In [127]:
df_toronto["Borough"] = df_toronto["Borough"].replace("Not assigned", np.nan)
df_toronto.dropna(subset=["Borough"],inplace=True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Combining Neighbourhood's that have the same Postcode seperated by ','

In [128]:
df_toronto_grouped = df_toronto.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ',' .join(x))
df_toronto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Neighbourhood's with 'Not assigned' are changed to the same name  as the Borough

In [129]:
nan_neigh_rows = df_toronto_grouped.Neighbourhood == 'Not assigned'
df_toronto_grouped.loc[nan_neigh_rows, 'Neighbourhood'] = df_toronto_grouped.loc[nan_neigh_rows, 'Borough']
df_toronto_grouped[nan_neigh_rows]

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


## Shape of the cleaned DataFrame

In [130]:
df_toronto_grouped.shape

(103, 3)

## Getting coordinates and adding them to the Toronto DataFrame

In [135]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coords = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [136]:
print(coors.shape)
coords.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the two DataFrames

In [138]:
df_toronto_temp = df_toronto_grouped.set_index('Postcode')
coords_temp = coords.set_index('Postal Code')
df_toronto_coords = pd.concat([df_toronto_temp, coords_temp], axis=1, join='inner')

## Resetting the Index

In [140]:
df_toronto_coords.index.name = 'PostalCode'
df_toronto_coords.reset_index(inplace=True)

## Final DataFrame with added coordinates

In [142]:
print(df_toronto_coords.shape)
df_toronto_coords.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
